# ESG analysis using Agents

In this example, we demonstrate how a generative AI agent can be used to perform ESG analysis. We will use two tools:
1. A relational database, which contains the ESG metric data for companies. Here we are using SQLite database (in-memory) with SQLAlchemy as the ORM to interact with the database.
2. A vector store which contains annual ESG report data for companies. Here we are using Pinecone serverless.


For this example, we are using sample ESG data for fictional companies with the name Company1, Company2 and Company3. ESG reports can be found under ESG-reports folder.

# Install the requirements

The first steps is to install the required libraries in the environment.

In [ ]:
%pip install -r requirements.txt

# Import

Next, we add the neccessary imports

In [ ]:
from uuid import uuid4
import time
from datetime import datetime

from sqlalchemy import MetaData
from sqlalchemy import Column, Integer, String, Table, Date, Float
from sqlalchemy import create_engine
from sqlalchemy import insert

from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_pinecone import PineconeVectorStore

from langchain import hub
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.documents import Document

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

from langchain.agents import Tool

# Setup the LLM and embedding model

Now we set up the LLM and embedding model.

For LLM, we are using Anthropic's Claude 3.5 Sonnet. Our initial tests showed that Llama3 was not producing satisfactory results with ReAct prompting whereas Claude3.5 Sonnet performed the task really well.

For embedding model, we are using the open source Hugging Face Sentence-Transformer embedding model

In [ ]:
DEFAULT_MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"


from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model=DEFAULT_MODEL_ID,
    temperature=0,
    max_tokens=None
)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Setting up in memory RDBMS

Now we set up the first tool which is RDBMS. We first create the in-memory SQL engine based on SQLite and then create a table ESG_DATA

In [3]:
# setting up in memory RDBMS with Sqqlite and SqlAlchemy as an ORM wrapper
#testing only the DB interaction in isolation for now...
sql_engine = create_engine("sqlite:///:memory:")
metadata = MetaData()

esg_data = Table(
    "esg_data",
    metadata,
    Column("instance_id", Integer, primary_key=True),
    Column("company_name", String(4), nullable=False),
    Column("esg_score", Float, nullable=False),
    Column("total_emmisions", Float, nullable=True),
    Column("net_zero_target", Date, nullable=True),
    Column("renewable_energy_pct", Float, nullable=True),
    Column("board_diversity_pct", Float, nullable=True),
)

metadata.create_all(sql_engine)

# Insert data in ESG_DATA table

Next we insert some sample data in ESG table using SQLAlchemy ORM wrapper.

In [4]:
records = [
    [1, 'Company1', 500, 600, datetime(2030,2,1), 40, 50],
    [2, 'Company2', 900, 700, datetime(2040,2,15), 30, 60],
    [3, 'Company3', 300, 300, datetime(2030, 3,15), 60, 40]
]

def insert_records(records):
    stmt = insert(esg_data).values(
    instance_id=records[0],
    company_name=records[1],
    esg_score=records[2],
    total_emmisions=records[3],
    net_zero_target=records[4],
    renewable_energy_pct=records[5],
    board_diversity_pct=records[6]
    )

    with sql_engine.begin() as conn:
        conn.execute(stmt)


for record in records:
    insert_records(record)

## Check if records are successfully persisted

As a test, we check if the table is correctly populated with the data or not. This is an optional step.

In [ ]:
s = esg_data.select()
conn = sql_engine.connect()
result = conn.execute(s)

for row in result:
   print (row)

# Build the database tools, to be used by Agent later

Now we prepare the database tool. Note that Langchain provides Toolkit for interacting with RDBMS which contains several tools for fetching data, checking schema etc. We will fetch the pre-built tools from the Toolkit .

In [ ]:
db = SQLDatabase(sql_engine)

sql_db_toolkit = SQLDatabaseToolkit(db=db, llm=llm)

db_tools = sql_db_toolkit.get_tools()

db_tools

## Set up Vector store

Now, we set up the vector store. Here we are using Pinecone serverless. To use this, you need to register on [pinecone website](app.pinecone.io) and get a free API key. This key can be used to create up to 5 indexes with enough storage to run this example. Replace the placeholder to put your pinecone key.

In [7]:
pinecone_key = "<<enter pinecone API key>>"

# configure client
pinecone_client = Pinecone(api_key=pinecone_key)

## Optional: Delete any pre-existing indexes 

If you are running this notebook multiple times, run the cell below to delete any pre-existing indexes as Pinecone free serverless edition has a limit of 5 indexes only.

In [8]:
indexes = pinecone_client.list_indexes()

vv = [pinecone_client.delete_index(i.name) for i in indexes] 

## Create the vector store index

Now, we create the vector store index. Note that the dimension of the vector store has to match the dimension of the embedding model which is 768 in this case. We are using cosine similarity as the vector search algorithm.

In [ ]:
#create the empty index
index_name = "esg-001"
existing_indexes = [
    index_info["name"] for index_info in pinecone_client.list_indexes()
]

# check if index already exists
if index_name not in existing_indexes:
    # if does not exist, create index
    pinecone_client.create_index(
        index_name,
        dimension=768,  # dimensionality of embedding model used
        metric='cosine', # or can also try dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    # wait for index to be created
    while not pinecone_client.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect
vector_index = pinecone_client.Index(index_name)
time.sleep(3)

vector_store = PineconeVectorStore(index=vector_index, embedding=embedding_model)

# check index stats -- should be empty for now
vector_index.describe_index_stats()

## Load documents in the vector store

Now, we load the documents in the vector store. The ESG reports are present under ESG-reports folder and we load each report one-by-one, chunk it and store the embedding in vector store.

We have kept a chunk size of 100 tokens with chunk overlap of 10%. This worked well for these reports. For a different document type, chunking size might need to be changed.

similarly, we are using RecursiveCharacterTextSplitter to chunk the document as it worked best for our reports. Based on the document type, you may want to try other splitter classes from Langchain.

In [10]:
# Load ESG documents

def add_document_to_vector_store(document_path):
    with open(document_path) as f:
        esg_report = f.read()

        text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size=100,
        chunk_overlap=10,
        length_function=len,
        is_separator_regex=False,
    )
    texts = text_splitter.split_documents([Document(esg_report)])

    uuids = [str(uuid4()) for _ in range(len(texts))]

    vector_store.add_documents(documents=texts, ids=uuids)


add_document_to_vector_store("ESG-reports/Company1-ESG-report.txt")
add_document_to_vector_store("ESG-reports/Company2-ESG-report.txt")
add_document_to_vector_store("ESG-reports/Company3-ESG-report.txt")

## Check if the documents are indexed

We just describe the index to verify if its filled with new vectors or not. The vector count should come around 337.

In [ ]:
vector_index.describe_index_stats()

## Setup the Vector store tool

Final step in vector store set up is to create the tool representation of the vector store so it can be supplied to the agent.

In [12]:
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# retrieval qa chain
vector_store_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

# Add tools to the agent

In this step, we merge both the database and vector store tools in a single tool list.

In [13]:

rag_tools = [
    Tool(
        name = 'VectorStore',
        func=vector_store_retriever.run,
        description="Useful for searching information from the knowledge base. This should be given the first priority while searching for information. It has information about Company1, Company2 and Company3's ESG reports in detail. If the information is not found, then the database tools must be used to find the answer"
    )
]

tools = rag_tools + db_tools

#only needed for Anthropic model
llm_with_tools = llm.bind_tools(tools)


# Setup and invoke agent
 Now we can set up the agent and invoke it with a query.

In [ ]:
from langchain.agents import AgentExecutor
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langsmith import Client

# use a prompt template, change it as per use-case.
prompt = hub.pull("hwchase17/structured-chat-agent")

# Construct the Tool Calling Agent
agent = create_structured_chat_agent(llm_with_tools, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# provide additional instruction to the agent to leverage all the tools for finding the right answer, before giving up.
additional_instructions_for_agent = "Use all the tools at your disposal to find the right answer for the query below: \n\n"

# Query
query = additional_instructions_for_agent + "What is the favorability rating on work-life balance and flexibility for Company1 in the year 2022??"

#invoke the agent
agent_executor.invoke({"input": query})

# Conclusion

This concludes the agentic implementation of ESG analysis.